<a href="https://colab.research.google.com/github/Anushka03716/DataScience/blob/main/ProjectDataScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from scipy.stats import boxcox
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline


In [ ]:
# Set the resolution of the plots
plt.rcParams['figure.dpi'] = 200

# Configure Seaborn plot styles
sns.set(rc={'axes.facecolor': '#faded9'}, style='darkgrid')

# Load dataset
df = pd.read_csv("heart.csv")

# Display the first few rows
print(df.head())

# Display dataframe info
df.info()

# Define continuous features
continuous_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']

# Convert remaining features to object (categorical)
feature_to_convert = [col for col in df.columns if col not in continuous_features]
df[feature_to_convert] = df[feature_to_convert].astype('object')

print(df.dtypes)

# Summary statistics
df.describe().T
df.describe(include=['object'])

# Continuous dataframe
df_continuous = df[continuous_features]

# Create subplots (2 rows × 3 columns)
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
axes = axes.flatten()

# Plot histograms
for i, col in enumerate(df_continuous.columns):

    values, bin_edges = np.histogram(
        df_continuous[col],
        range=(
            np.floor(df_continuous[col].min()),
            np.ceil(df_continuous[col].max())
        )
    )

    graph = sns.histplot(
        data=df_continuous,
        x=col,
        bins=bin_edges,
        kde=True,
        ax=axes[i],
        edgecolor='none',
        color='#69b3a2',
        alpha=0.7,
        line_kws={'lw': 3}
    )

    axes[i].set_xlabel(col, fontsize=12)
    axes[i].set_ylabel('Count', fontsize=12)
    axes[i].set_xticks(np.round(bin_edges, 1))
    axes[i].set_xticklabels(axes[i].get_xticks(), rotation=45)
    axes[i].grid(color='lightgray')

    for j, p in enumerate(graph.patches):
        axes[x,y].annotate('{}'.format(int(p.get_height())),
         (p.get_x() + p.get_width() / 2., p.get_height() + 1), ha='center', fontsize=10)

# Hide unused subplot
axes[5].axis('off')

plt.suptitle('Distribution of Continuous Features', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()


In [ ]:
# Mean & Std text box (UNCHANGED LOGIC)
    textstr = '\n'.join((
        r'$\mu=%.2f$' % df_continuous[col].mean(),
        r'$\sigma=%.2f$' % df_continuous[col].std()
    ))

    ax[i].text(
        0.75, 0.9, textstr,
        transform=ax[i].transAxes,
        fontsize=12,
        verticalalignment='top',
        color='white',
        bbox=dict(
            boxstyle='round',
            facecolor='#ff826e',
            edgecolor='white',
            pad=0.5
        )
    )

ax[5].axis('off')  # Hide empty subplot
plt.suptitle('Distribution of Continuous Features', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()

#Filter out categorical features for the univariate analysis
categorical_features = df.columns.difference(continuous_features)
df_categorical = df[categorical_features]

# Set up the subplot for a 4x2 layout
fig, ax = plt.subplots(nrows=4, ncols=2, figsize=(15, 20))

# Loop through categorical features and create count plots
for i, col in enumerate(categorical_features):
    row = i // 2
    col_idx = i % 2

    #Calculate frequency percentges
    value_counts = df[col].value_counts(normalize=True).mul(100).sort_values()

    #plot bar chart
    value_counts.plot(kind='barh', ax=ax[row, col_idx], color='#69b3a2', edgecolor='none', width=0.8)

    for index, value in enumerate(value_counts):
        ax[row, col_idx].text(
            value + 1, index, f'{value:.1f}%',
            va='center', fontsize=10, fontweight='bold'
        )

    ax[row, col_idx].set_title(col, fontsize=12)
    ax[row, col_idx].set_xlabel('Percentage', fontsize=10)
    ax[row, col_idx].set_xlim([0,95])

ax[4,1].axis('off')  # Hide the empty subplot
plt.suptitle('Distribution of Categorical Features', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.subplots_adjust(top=0.95)
plt.show()

#Set color palette
sns.set_palette(['#ff826e', '#69b3a2'])

# Create the subplots
fig, ax = plt.subplots(len(continuous_features), 2, figsize=(15, 20), gridspec_kw={'width_ratios': [3, 1]})

#loop through each continuos features to create barplots and kde plots
for i, col in enumerate(continuous_features):
    #Barplot showing the mean value of the feature for eac target category
    graph = sns.barplot(data=df, x="target", y=col, ax=ax[i,0])

    #KDE plot showing the distribution of the feature for each target category
    sns.kdeplot(data=df[df["target"]==0], x=col, fill=True, linewidth=2, ax=ax[i,1], label='0')
    sns.kdeplot(data=df[df["target"]==1], x=col, fill=True, linewidth=2, ax=ax[i,1], label='1')
    ax[i,1].set_ytricks([])
    ax[i,1].legend(title='Heart Diease', loc='upper right')

    #Add mean values to the barplot
    for cont in graph.containers:
        graph.bar_label(cont, fmt='     %3g')

# Set titles and labels
plt.suptitle('Continuous Features vs Target Variable', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

#Remove 'target' from the list of categorical features for the count plot
categorical_features = [feature for feature in categorical_features if feature != 'target']

fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15, 10))

for i, col in enumerate(categorical_features):
    #create a cross-tabulation showing the proportion of purchased and non-purchased loans for each category
    cross_tab = pd.crosstab(index=df[col], columns=df['target'])
    #Using the normalize = True argument given us the index-wise proportion of the data
    cross_tab_prop = pd.crosstab(index=df[col], columns=df['target'], normalize='index')

    #Define colormap
    cmp = ListedColormap(['#ff826e', '#69b3a2'])

    #Plot stacked bar chart
    cross_tab_prop.plot(kind='bar', ax=ax[x,y], stacked=True, width=0.8, colormap=cmp, legend=False, ylabel='Proportion',sharey=True)

    #Add the proportion and counts of the individual bars to the plot
    for idx, val in enumerate([*cross_tab.index.values]):
        for(proportion, count, y_location) in zip(cross_tab_prop.loc[val], cross_tab.loc[val], cross_tab.loc[val],cross_tab_prop.loc[val].cumsum()):
            ax[x,y].text(x=idx-0.3, y=(y_location - proportion) + (proportion/2)-0.3, s=f' {count}\n{np.round(proportion * 100,1)}%)', color = "black", fontsize=9, fontweight='bold')

    #Add legend
    ax[x,y].legend(title='target', loc=(0.7, 0.9), fontsize=8, ncol=2)
    #Set y limit
    ax[x,y].set_ylim([0,1.12])
    #Rotate x-ticks
    ax[x,y].set_xticklabels(ax[x,y].get_xticklabels(), rotation=0)

plt.suptitle('Categorical Features vs Target Variable', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#Checking for missing values
print(df.isnull().sum())


In [ ]:
continuous_features

In [ ]:
q1 = df[continuous_features].quantile(0.25)
q3 = df[continuous_features].quantile(0.75)
iqr = q3 - q1
outliers_count_specified = ((df[continuous_features] < (q1 - 1.5 * iqr)) | (df[continuous_features] > (q3 + 1.5 * iqr))).sum()
print(outliers_count_specified)


In [ ]:
#Implementing one-hot encoding on the specified categorical features
df_encoded = pd.get_dummies(df, columns=['cp', 'restecg', 'thal'], drop_first=True)

#Convert the rest of the categorical variables that dont need one-hot encoding
features_to_convert = ['sex', 'fbs', 'exang', 'slope', 'ca', 'target']
for feature in features_to_convert:
    df_encoded[feature] = df_encoded[feature].astype('int')
df_encoded.dtypes

In [ ]:
df_encoded.head()

In [ ]:
#Define features (X) and output (y)
X = df_encoded.drop('target', axis=1)
y = df_encoded['target']

#Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

In [ ]:
continuous_features

In [ ]:
#Adding a small constant to 'oldpeak' to avoid zero values for Box-Cox transformation
X_train['oldpeak'] = df_encoded['oldpeak'] + 0.01
X_test['oldpeak'] = df_encoded['oldpeak'] + 0.01

#Checking the distribution of the continuous features
fig, ax = plt.subplots(2, 5, figsize=(15, 10))

#Original distribution
for i, col in enumerate(continuous_features):
    sns.histplot(X_train[col], kde=True, ax=ax[0, i], color='#69b3a2', edgecolor='none', alpha=0.7, line_kws={'lw': 3})
    ax[0, i].set_title(f'Original {col}', fontsize=10)
    ax[0, i].set_xlabel('')
    ax[0, i].set_ylabel('')

In [ ]:
lambdas = {}

for i, col in enumerate(continuous_features):
    #Only apply box-cox +ve val
    if X_train[col].min() > 0:
        X_train[col], lambdas[col] = boxcox(X_train[col])
        #Applying the same lambda to test data
        X_test[col] = boxcox(X_test[col], kde=True, ax=ax[1,i], color = 'red').set_title(f'Transformed'{})
    else:
        sns.histplot(X_train[col], kde=True, ax=ax[1,i], color='green')set_title(f'{col(Not)}')

fig.tight_layout()
plt.show()